In [9]:
import pandas as pd
import numpy as np
import operator
import string
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.data import load
from nltk.stem import SnowballStemmer
from string import punctuation
import re
from sklearn.utils import shuffle

#LEER EL DATASET
data = pd.read_csv("Dataset5.csv", sep=",")
shuffle(data)
##########################################
punctuation = list(string.punctuation)
stemmer = SnowballStemmer("spanish")
stop = stopwords.words('spanish') + punctuation
vc = TfidfVectorizer()
Tweet = data["Tweet"]
Label = data["Label"]
T=np.asarray(Tweet)

########################################
emoticons_str = r"""
    (?:
        [:=;] # :(
        [oO\-]? # No se
        [D\)\]\(\]/\\OpP] # 
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)', # anything else
    r'[0-9]+'
]
##############Metodos####################

####Quitar numeros##############
def remove(list): 
    pattern = '[0-9]'
    list = [re.sub(pattern, '', i) for i in list]
    return list
#######################################
def generate_ngrams(s, n):
    # Convert to lowercases
    s = s.lower()
    # Romper la oración en el tokens
    tokens_first = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE )
    tokens = tokens_first.findall(s)
        
    # eliminar tokens vacíos, links, menciones, stopwords y hastags
    tokens = [token for token in tokens if (token != "" or token != '' ) and (token not in stop) and (not token.startswith(('#', '@', 'https:', 'http:','<')))]
    #lematizar
    tokens = [stemmer.stem(token) for token in tokens]
    tokens = remove(tokens)
    tokens = [token for token in tokens if (token != "" or token != ''  ) and (token not in stop) and (not token.startswith(('#', '@', 'https:', 'http:')))]
#     print(tokens)
    # funcion zip genera n-grams
    # Concatena los tokens en los ngrams y los retorna
    ngrams = zip(*[tokens[i:] for i in range(n)])
    return [" ".join(ngram) for ngram in ngrams]

def computeTF(wordDict, bagOfWords):
    tfDict = {}
    bagOfWordsCount = len(bagOfWords)
    for word, count in wordDict.items():
        if bagOfWordsCount != 0:
            tfDict[word] = count / float(bagOfWordsCount)
        else:
            tfDict[word] = 0

    return tfDict


def computeIDF(documents):
    import math
    N = len(documents)
    idfDict = dict.fromkeys(documents[0].keys(), 0)
    
    for document in documents:
        for word, val in document.items():
            
            if val > 0:
                idfDict[word] += 1
    for word, val in idfDict.items():
        if val != 0:
            idfDict[word] = math.log(N / float(val))
        else:
            idfDict[word] = 0
    return idfDict

def computeTFIDF(tfBagOfWords, idfs):
    tfidf = {}
    for word, val in tfBagOfWords.items():
        tfidf[word] = val * idfs[word]
    return tfidf

#########################################


In [13]:
#############Documentos##################
bigNumOfWords = []
bigBagOfWords = []
unique = []
bigTf = {}
bigTfidf= []
##########Preprocesamiento##########
n=2
count = 0
count2 = 0
for tweet in T:
    bagOfWords = generate_ngrams(str(tweet), n)
    unique = set(unique).union(set(bagOfWords))
    bigBagOfWords.append(bagOfWords)
############Contar Palabras######################
for tweet in bigBagOfWords:
    numOfWords = dict.fromkeys(unique, 0)
    for word in tweet:
        numOfWords[word] += 1
    bigNumOfWords.append(numOfWords)

count = 0
##################TF##########################
for tweet in T:
    tf = computeTF(bigNumOfWords[count], bigBagOfWords[count])
    bigTf[count] = tf
    count += 1
#####################IDF###################
idfs = computeIDF(bigNumOfWords)
###################TFIDF###################

for tweet in T:
    tfidf = computeTFIDF(bigTf[count2], idfs)
    bigTfidf.append(tfidf)
    count2 += 1
#################results############################
df = pd.DataFrame(bigTfidf)
df['Label'] = Label
# print(df['Label'])
if n == 1:
    df.to_csv (r'Features unigrama.csv', index = False, header=True)
else:
    df.to_csv (r'features bigram.csv', index = False, header=True)

In [8]:
print(bigBagOfWords[0])
print(bigBagOfWords[1])

print(df['Label'][15])

['sali', 'notici', 'mam', 'habl', 'dia', 'dij', 'jep', 'import']
['sali', 'notici', 'mam', 'habl', 'dia', 'dij', 'jep', 'import']
1


In [51]:

n = [" ".join(ngram) for ngram in bigBagOfWords]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(n)
vocabulary = vectorizer.get_feature_names()
df['Label'] = Label
df = pd.DataFrame(data=X.toarray(), columns=vocabulary).iloc[:,0::2]
df['Label'] = Label
print(df)

     _no  abaj  abandon  abiert  abort  abri  abrim  absoluci  absolv  aburri  \
0    0.0   0.0      0.0     0.0    0.0   0.0    0.0       0.0     0.0     0.0   
1    0.0   0.0      0.0     0.0    0.0   0.0    0.0       0.0     0.0     0.0   
2    0.0   0.0      0.0     0.0    0.0   0.0    0.0       0.0     0.0     0.0   
3    0.0   0.0      0.0     0.0    0.0   0.0    0.0       0.0     0.0     0.0   
4    0.0   0.0      0.0     0.0    0.0   0.0    0.0       0.0     0.0     0.0   
..   ...   ...      ...     ...    ...   ...    ...       ...     ...     ...   
728  0.0   0.0      0.0     0.0    0.0   0.0    0.0       0.0     0.0     0.0   
729  0.0   0.0      0.0     0.0    0.0   0.0    0.0       0.0     0.0     0.0   
730  0.0   0.0      0.0     0.0    0.0   0.0    0.0       0.0     0.0     0.0   
731  0.0   0.0      0.0     0.0    0.0   0.0    0.0       0.0     0.0     0.0   
732  0.0   0.0      0.0     0.0    0.0   0.0    0.0       0.0     0.0     0.0   

     ...  voz  vuelt  web  

In [53]:
n = [" ".join(ngram) for ngram in bigBagOfWords]
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(n)
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)
print(df)

     _no  aaaaayyyy  abaj  abam  abandon  aberr  abiert  abol  abort  abraz  \
0    0.0        0.0   0.0   0.0      0.0    0.0     0.0   0.0    0.0    0.0   
1    0.0        0.0   0.0   0.0      0.0    0.0     0.0   0.0    0.0    0.0   
2    0.0        0.0   0.0   0.0      0.0    0.0     0.0   0.0    0.0    0.0   
3    0.0        0.0   0.0   0.0      0.0    0.0     0.0   0.0    0.0    0.0   
4    0.0        0.0   0.0   0.0      0.0    0.0     0.0   0.0    0.0    0.0   
..   ...        ...   ...   ...      ...    ...     ...   ...    ...    ...   
728  0.0        0.0   0.0   0.0      0.0    0.0     0.0   0.0    0.0    0.0   
729  0.0        0.0   0.0   0.0      0.0    0.0     0.0   0.0    0.0    0.0   
730  0.0        0.0   0.0   0.0      0.0    0.0     0.0   0.0    0.0    0.0   
731  0.0        0.0   0.0   0.0      0.0    0.0     0.0   0.0    0.0    0.0   
732  0.0        0.0   0.0   0.0      0.0    0.0     0.0   0.0    0.0    0.0   

     ...  yegu  yend  yidis  zalamer  zon  zuluag  

In [34]:

test = ['abaj voz web jep zon', 'abaj jep abiert zon', 'abiert abiert']
X_test = vectorizer.transform(test)
pd.DataFrame(data=X_test.toarray(), columns=vocabulary).iloc[:, 0::2]

,_no,abaj,abandon,abiert,abort,abri,abrim,absoluci,absolv,aburri,...,volvi,voz,vuelt,web,xa,yegu,yidis,zon,zurd,ñia
0,0.0,0.504708,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.475073,0.0,0.504708,0.0,0.0,0.0,0.504708,0.0,0.0
1,0.0,0.598478,0.0,0.519064,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.598478,0.0,0.0
2,0.0,0.000000,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0


In [27]:
data = pd.read_csv("Features unigrama.csv", sep=";", low_memory=False)
dataset = list(data)
print(dataset)

['bassil', 'transicional', 'das', 'timochenk', 'culp', 'sol', 'engan', '500', 'respuest', 'extravi', 'cint', 'arrodill', 'intermedi', 'fumigacion', 'artist', 'pinch', 'b', 'banquer', 'dialogonacional', 'apuest', 'person', 'cultiv', '6,748', 'anuenci', 'aprend', 'deshonest', 'jefesit', '15', 'holm', 'aval', 'cas', 'parasit', 'claroo', 'polit', 'imperfect', 'pts', 'alejandr', 'vim', 'despreci', 'ajust', 'adopt', 'impresion', 'fuerz', 'ves', 'beb', 'absolut', 'curs', 'alto', 'quep', 'prest', 'ataqu', 'santamari', 'petrism', 'exist', 'definit', 'pit', 'asom', 'prostitu', 'dec', 'hac', 'expedient', 'ambiental', 'rubr', 'epa', 'aument', 'licenci', 'empujon', 'obten', 'ministr', 'robl', 'vean', 'quej', 'infernal', 'elector', 'tipificacion', 's', 'pendient', 'reintegr', 'mandader', 'extraordinari', 'sexual', 'financi', 'agreg', 'orejon', 'picot', 'nue', 'valient', 'arredond', 'impun', 'elect', 'bandal', 'direct', 'etern', 'extrajudicial', 'indemniz', 'cinism', 'porq', 'destroz', 'ingres', 'men

In [42]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB

dataset = pd.read_csv("TFIDF.csv", sep=",", low_memory=False)

y = dataset.Label
X = dataset.drop('Label', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.25)
print("\nX_train:\n")
print(X_train.head())
print(X_train.shape)

print("\nX_test:\n")
print(X_test.head())
print(X_test.shape)


X_train:

      Unnamed: 0  semblanz  ifueput  propon  divers  ntra  aclaraci  especial  \
1916        1916       0.0      0.0     0.0     0.0   0.0       0.0       0.0   
1132        1132       0.0      0.0     0.0     0.0   0.0       0.0       0.0   
3760        3760       0.0      0.0     0.0     0.0   0.0       0.0       0.0   
940          940       0.0      0.0     0.0     0.0   0.0       0.0       0.0   
2839        2839       0.0      0.0     0.0     0.0   0.0       0.0       0.0   

      increment  rar  ...  ofici  explos  nbq  farsapaz  cur  hab  inconform  \
1916        0.0  0.0  ...    0.0     0.0  0.0       0.0  0.0  0.0        0.0   
1132        0.0  0.0  ...    0.0     0.0  0.0       0.0  0.0  0.0        0.0   
3760        0.0  0.0  ...    0.0     0.0  0.0       0.0  0.0  0.0        0.0   
940         0.0  0.0  ...    0.0     0.0  0.0       0.0  0.0  0.0        0.0   
2839        0.0  0.0  ...    0.0     0.0  0.0       0.0  0.0  0.0        0.0   

      reg  crecimient

In [44]:
#Defino el algoritmo a utilizar
#Naive Bayes
from sklearn.naive_bayes import GaussianNB
algoritmo = GaussianNB()
#Entreno el modelo
algoritmo.fit(X_train, y_train)
#Realizo una predicción
y_pred = algoritmo.predict(X_test)
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)
#Calculo la precisión del modelo
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred,average='micro')
print('Precisión del modelo:')
print(precision)

Matriz de Confusión:
[[ 61 358  57]
 [  4  49   1]
 [ 12 368 108]]
Precisión del modelo:
0.21414538310412573
